# Reference
###  https://www.kaggle.com/yuyougnchan/tps-nov-lightgbm-baseline/notebook
### https://www.kaggle.com/sergiosaharovskiy/tps-nov-2021-a-complete-guide

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier, RandomForestRegressor
from sklearn.calibration import calibration_curve
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_curve, RocCurveDisplay, ConfusionMatrixDisplay, confusion_matrix, roc_auc_score, accuracy_score
from sklearn.model_selection import GridSearchCV,train_test_split

from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
import gc

In [3]:
DATASET_PATH ="./dataset/"
train = pd.read_csv(DATASET_PATH + "train.csv")
test = pd.read_csv(DATASET_PATH + "test.csv")

In [4]:
train.isnull().sum()[train.isnull().sum() != 0 ]

Series([], dtype: int64)

In [5]:
train_df = train.drop(['id'],axis=1)
test_df = test.drop(['id'],axis=1)

In [6]:
#test_df

In [7]:
X = train_df.drop('target', axis=1)
Y = train_df['target']

X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2,
                                                     shuffle=True, random_state=0)

In [8]:
cols = ['f'+str(i) for i in range(100)]

In [9]:
X

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
0,0.106643,3.594370,132.8040,3.184280,0.081971,1.18859,3.732380,2.266270,2.099590,0.012330,...,0.010739,1.098620,0.013331,-0.011715,0.052759,0.065400,4.211250,1.978770,0.085974,0.240496
1,0.125021,1.673360,76.5336,3.378250,0.099400,5.09366,1.275620,-0.471318,4.545940,0.037706,...,0.135838,3.460170,0.017054,0.124863,0.154064,0.606848,-0.267928,2.577860,-0.020877,0.024719
2,0.036330,1.497470,233.5460,2.194350,0.026914,3.12694,5.056870,3.849460,1.801870,0.056995,...,0.117310,4.883000,0.085222,0.032396,0.116092,-0.001688,-0.520069,2.141120,0.124464,0.148209
3,-0.014077,0.246000,779.9670,1.890640,0.006948,1.53112,2.698000,4.517330,4.503320,0.123494,...,-0.015347,3.474390,-0.017103,-0.008100,0.062013,0.041193,0.511657,1.968600,0.040017,0.044873
4,-0.003259,3.715420,156.1280,2.147720,0.018284,2.09859,4.154920,-0.038236,3.371450,0.034166,...,0.013781,1.910590,-0.042943,0.105616,0.125072,0.037509,1.043790,1.074810,-0.012819,0.072798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,0.043008,1.640550,1375.8900,1.087030,0.839580,1.69362,3.843520,1.108250,1.505390,-0.022177,...,0.046537,1.552280,0.080317,0.017643,0.225133,0.073179,4.436510,1.739330,0.049038,0.065804
599996,0.137048,4.826860,-99.6478,0.985289,0.037712,3.30370,2.471790,5.942130,0.875148,-0.009738,...,0.109293,1.345210,0.092457,0.081926,0.234041,0.041383,3.873990,1.870100,0.162313,0.085662
599997,1.316220,0.502062,66.2360,2.233020,0.060038,3.77371,2.655340,3.155890,1.055020,0.025137,...,0.117131,2.181150,0.032708,0.109668,0.027466,0.061931,-0.383329,0.922113,0.084864,0.052635
599998,0.511918,3.827470,777.8720,0.285914,0.520157,2.76285,2.522200,1.561360,1.971140,0.461982,...,0.072443,0.366702,0.070943,0.071452,-0.038071,0.038829,-0.122888,3.107790,0.061333,0.004179


In [10]:
scaler = StandardScaler()
X[cols] = scaler.fit_transform(X[cols])
X

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
0,-0.382553,0.705772,-0.315075,0.347277,-0.229657,-0.875660,0.660314,-0.197064,-0.286162,-0.289270,...,-0.537157,-0.872508,-0.258806,-0.595537,-0.199502,-0.196145,1.067358,-0.400887,-0.167145,0.443374
1,-0.347377,-0.530387,-0.417061,0.472862,-0.187909,1.623543,-0.910506,-1.963980,1.309644,-0.229122,...,0.573313,0.658473,-0.252018,0.548089,0.019765,2.392938,-1.806811,-0.008064,-0.412110,-0.371198
2,-0.517136,-0.643571,-0.132486,-0.293650,-0.361533,0.364863,1.507175,0.824771,-0.480372,-0.183401,...,0.408845,1.580886,-0.127714,-0.226174,-0.062423,-0.516946,-1.968603,-0.294434,-0.078904,0.094984
3,-0.613619,-1.448884,0.857867,-0.490286,-0.409357,-0.656445,-0.001055,1.255833,1.281843,-0.025780,...,-0.768719,0.667692,-0.314304,-0.565262,-0.179472,-0.311897,-1.306572,-0.407556,-0.272505,-0.295118
4,-0.592913,0.783666,-0.272802,-0.323841,-0.382205,-0.293270,0.930480,-1.684456,0.543499,-0.237512,...,-0.510155,-0.346112,-0.361423,0.386926,-0.042986,-0.329511,-0.965117,-0.993613,-0.393636,-0.189697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,-0.504355,-0.551500,1.937940,-1.010581,1.585028,-0.552446,0.731375,-0.944482,-0.673772,-0.371060,...,-0.219391,-0.578403,-0.136659,-0.349704,0.173589,-0.158944,1.211901,-0.557888,-0.251825,-0.216101
599996,-0.324356,1.498871,-0.736379,-1.076453,-0.335670,0.477988,-0.145691,2.175439,-1.084892,-0.341576,...,0.337680,-0.712645,-0.114522,0.188558,0.192870,-0.310986,0.850947,-0.472142,0.007868,-0.141134
599997,1.932651,-1.284110,-0.435725,-0.268614,-0.282193,0.778789,-0.028331,0.377122,-0.967558,-0.258913,...,0.407256,-0.170709,-0.223473,0.420855,-0.254249,-0.212734,-1.880861,-1.093736,-0.169690,-0.265813
599998,0.393168,0.855770,0.854070,-1.529262,0.819921,0.131849,-0.113459,-0.652033,-0.369953,0.776531,...,0.010571,-1.347007,-0.153752,0.100857,-0.396098,-0.323202,-1.713743,0.339410,-0.223636,-0.448741


In [11]:
def rmse(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))

In [12]:
def get_best_params(model, params):
    grid_search = GridSearchCV(model, param_grid=params, cv=3)
    grid_search.fit(X_train, y_train)
    print('best parameters : ', grid_search.best_params_)
    print('best score : ', grid_search.best_score_)
    y_pred = grid_search.predict(X_valid)
    print('validation score ',accuracy_score(y_valid,y_pred))
    print('rmse score : ',rmse(y_valid,y_pred))
    return grid_search.best_params_

In [17]:
lgbm_params = { 'n_estimators' : [4100], 'colsample_bytree' : [0.35, 0.55,0.7],
              'max_depth' : [4,6,15], 'num_leaves' : [21, 31,41],
               'subsample' : [0.4, 0.6]
              }
lgbm_clf = LGBMClassifier( learning_rate=0.07, boosting_type='gbdt',
                           reg_lambda=5, n_jobs=10,
                          device="cuda"
                          
                         )
best_lgbm = get_best_params(lgbm_clf, lgbm_params)

best parameters :  {'colsample_bytree': 0.35, 'max_depth': 15, 'n_estimators': 4100, 'num_leaves': 21, 'subsample': 0.4}
best score :  0.7038875
validation score  0.7091833333333334
rmse score :  0.5392742035983797


In [18]:
best_lgbm

{'colsample_bytree': 0.35,
 'max_depth': 15,
 'n_estimators': 4100,
 'num_leaves': 21,
 'subsample': 0.4}

In [19]:
gc.collect()
#!stop here

360

In [49]:
def load_model():
    # rf : Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /media/2/hwbae0326/package/LightGBM/src/boosting/rf.hpp
    lgbm_clf = LGBMClassifier( boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.35, #feature_fraction=0.15, # same thing
               importance_type='split', learning_rate=0.1, max_depth=15,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=4100,  num_leaves=21, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=5, silent=True,
               #bagging_fraction=0.2,
               subsample=0.4, subsample_for_bin=200000,  subsample_freq=0,
                             n_jobs=10, device="cuda")
    return lgbm_clf

In [50]:
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=3, random_state=33, shuffle=True)
preds = np.zeros(test.shape[0])
auc = []
acc = []
n = 0

In [51]:
for train_idx, test_idx in kf.split(X_train[cols],y_train):
    xtrain, xval = X_train[cols].iloc[train_idx], X_train[cols].iloc[test_idx]
    ytrain, yval = y_train.iloc[train_idx], y_train.iloc[test_idx]
    
    model = load_model()
    model.fit(xtrain, ytrain, eval_set =[(xval,yval)], verbose=False)
    local_preds =  model.predict(test_df[cols]) # /kf.nsplits
    # print(local_preds, local_preds.shape)
    preds += local_preds
    auc.append(roc_auc_score(yval, model.predict_proba(xval)[:, 1]))
    acc.append(accuracy_score(yval, model.predict(xval)))

    print(f"fold: {n+1} , accuracy: {round(acc[n]*100,3)} , auc: {round(auc[n]*100,3)}")
    val_pred = model.predict(xval)
    print('rmse : %.4f'%(rmse(val_pred,yval)))
    n+=1

fold: 1 , accuracy: 69.686 , auc: 73.591
rmse : 0.5506
fold: 2 , accuracy: 69.363 , auc: 73.383
rmse : 0.5535
fold: 3 , accuracy: 69.535 , auc: 73.441
rmse : 0.5520


In [52]:
#lgbm_clf.fit(X_train, y_train)

In [53]:
kf = StratifiedKFold(n_splits=3, random_state=33, shuffle=True)
for train_idx, test_idx in kf.split(X_train[cols],y_train):
    xtrain, xval = X_train[cols].iloc[train_idx], X_train[cols].iloc[test_idx]
    ytrain, yval = y_train.iloc[train_idx], y_train.iloc[test_idx]
    
    model = LogisticRegression( n_jobs=10)
    model.fit(xtrain, ytrain)
    local_preds =  model.predict(test_df[cols]) # /kf.nsplits
    # print(local_preds, local_preds.shape)
    preds += local_preds
    auc.append(roc_auc_score(yval, model.predict_proba(xval)[:, 1]))
    acc.append(accuracy_score(yval, model.predict(xval)))

    print(f"fold: {n+1} , accuracy: {round(acc[n]*100,3)} , auc: {round(auc[n]*100,3)}")
    val_pred = model.predict(xval)
    print('rmse : %.4f'%(rmse(val_pred,yval)))
    n+=1

fold: 4 , accuracy: 70.547 , auc: 73.803
rmse : 0.5427
fold: 5 , accuracy: 70.158 , auc: 73.451
rmse : 0.5463
fold: 6 , accuracy: 69.57 , auc: 73.252
rmse : 0.5516


In [54]:
kf = StratifiedKFold(n_splits=3, random_state=33, shuffle=True)
for train_idx, test_idx in kf.split(X_train[cols],y_train):
    xtrain, xval = X_train[cols].iloc[train_idx], X_train[cols].iloc[test_idx]
    ytrain, yval = y_train.iloc[train_idx], y_train.iloc[test_idx]
    
    model = GradientBoostingClassifier() # XGBClassifier
    model.fit(xtrain, ytrain)
    local_preds =  model.predict(test_df[cols]) # /kf.nsplits
    # print(local_preds, local_preds.shape)
    preds += local_preds
    auc.append(roc_auc_score(yval, model.predict_proba(xval)[:, 1]))
    acc.append(accuracy_score(yval, model.predict(xval)))

    print(f"fold: {n+1} , accuracy: {round(acc[n]*100,3)} , auc: {round(auc[n]*100,3)}")
    val_pred = model.predict(xval)
    print('rmse : %.4f'%(rmse(val_pred,yval)))
    n+=1

fold: 7 , accuracy: 67.709 , auc: 71.952
rmse : 0.5682
fold: 8 , accuracy: 67.566 , auc: 71.751
rmse : 0.5695
fold: 9 , accuracy: 67.604 , auc: 71.745
rmse : 0.5692


In [55]:
kf = StratifiedKFold(n_splits=3, random_state=33, shuffle=True)
for train_idx, test_idx in kf.split(X_train[cols],y_train):
    xtrain, xval = X_train[cols].iloc[train_idx], X_train[cols].iloc[test_idx]
    ytrain, yval = y_train.iloc[train_idx], y_train.iloc[test_idx]
    
    model = XGBClassifier()
    model.fit(xtrain, ytrain)
    local_preds =  model.predict(test_df[cols]) # /kf.nsplits
    # print(local_preds, local_preds.shape)
    preds += local_preds
    auc.append(roc_auc_score(yval, model.predict_proba(xval)[:, 1]))
    acc.append(accuracy_score(yval, model.predict(xval)))

    print(f"fold: {n+1} , accuracy: {round(acc[n]*100,3)} , auc: {round(auc[n]*100,3)}")
    val_pred = model.predict(xval)
    print('rmse : %.4f'%(rmse(val_pred,yval)))
    n+=1

/media/2/hwbae0326/anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:35:35] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fold: 10 , accuracy: 68.821 , auc: 72.875
rmse : 0.5584


/media/2/hwbae0326/anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:36:20] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fold: 11 , accuracy: 68.742 , auc: 72.643
rmse : 0.5591


/media/2/hwbae0326/anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:37:04] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fold: 12 , accuracy: 68.859 , auc: 72.747
rmse : 0.5580


In [57]:
preds = preds / n
preds

array([1.        , 0.91666667, 1.        , ..., 0.58333333, 1.        ,
       0.91666667])

In [56]:
n

12

In [58]:
sub = pd.read_csv(DATASET_PATH+'/sample_submission.csv')
sub['target'] = preds
sub.to_csv('submission6.csv',index=False)
sub

,id,target
0,600000,1.000000
1,600001,0.916667
2,600002,1.000000
3,600003,0.750000
4,600004,0.166667
...,...,...
539995,1139995,1.000000
539996,1139996,1.000000
539997,1139997,0.583333
539998,1139998,1.000000


100%|██████████████████████████████████████| 11.5M/11.5M [00:04<00:00, 2.89MB/s]
400 - Bad Request


In [ ]:
!kaggle competitions submit -c tabular-playground-series-nov-2021 -f submission5.csv -m "LightGBM_7 + LogisticRegression_3 + GradientBoostingClassifier_7 KFold 10"
!kaggle competitions submit -c tabular-playground-series-nov-2021 -f submission6.csv -m "LightGBM_3 + LogisticRegression_3 + GradientBoostingClassifier_3 + XGBClassifier_3 KFold 10"
